# PowerGIM Example:
## Offshore grid optimisation (deterministic without uncertain paramters)




In [466]:
%matplotlib inline
import powergama
import powergama.powergim as pgim
import powergama.plots
import powergama.sampling
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as rnd
import copy
import lxml.etree
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [467]:
#specify random number seed so re-running the script gives the same result
rnd.seed(2016) 
reuse_sample = True

### Reading data and setting up optimisation model

In [468]:
grid_data = powergama.GridData()
grid_data.readSipData(nodes = "data/dog_nodes.csv",
                  branches = "data/dog_branches.csv",
                  generators = "data/dog_generators.csv",
                  consumers = "data/dog_consumers.csv")
file_parameters = 'data/dog_data_irpwind.xml'

# Profiles:
if reuse_sample:
    print("\n<> Loading time-series sample...")
    samplesize = 100
    grid_data.readProfileData(filename= "data/timeseries_sample_100_rnd2016.csv",
                              timerange=range(samplesize), timedelta=1.0)
else:
    # create new sample
    grid_data.readProfileData(filename = "data/timeseries_doggerbank.csv",
                              timerange = range(8760), timedelta = 1.0)

    print("\n<> Sampling...\n")
    samplingmethod = 'kmeans'
    samplesize = 3
    profiledata_sample = powergama.sampling.sampleProfileData(data=grid_data,
        samplesize=samplesize,sampling_method=samplingmethod)
    #profiledata_sample.to_csv("data/timeseries_sample_100_rnd2016.csv")
    grid_data.timerange = range(profiledata_sample.shape[0])
    grid_data.profiles = profiledata_sample




<> Loading time-series sample...


In [469]:
displayData = True
num_lines = 30
def xml_to_df(elements):
    data = [el.attrib.values() for el in elements]
    df=pd.DataFrame(data,columns=elements[0].keys())
    return df
if displayData:
    x = lxml.etree.parse(file_parameters)
    print("PARAMETERS - nodetype:")
    display(xml_to_df(x.getroot().xpath("nodetype/*")).set_index('name'))
    print("PARAMETERS - branchtype:")
    display(xml_to_df(x.getroot().xpath("branchtype/*")).set_index('name'))
    print("PARAMETERS - gentype:")
    display(xml_to_df(x.getroot().xpath("gentype/*")).set_index('name'))
    print("PARAMETERS - parameters:")
    display(xml_to_df(x.getroot().xpath("parameters")))
    print("\nNodes:")
    display(grid_data.node.head(num_lines))
    print("Branches:")
    display(grid_data.branch.head(num_lines))
    print("Consumers:")
    display(grid_data.consumer.head(num_lines))
    print("Generators:")
    display(grid_data.generator.head(num_lines))
    print("Profiles:")
    display(grid_data.profiles.head(num_lines))

PARAMETERS - nodetype:


,L,S
name,,
ac,1,50e6
dc,1,1


PARAMETERS - branchtype:


,B,Bdp,Bd,CL,CLp,CS,CSp,maxCap,lossFix,lossSlope
name,,,,,,,,,,
ac,5000e3,1.15e3,656e3,1562e3,0,4813e3,0,400,0,5e-5
dcmesh,5000e3,0.47e3,680e3,0,0,0,0,2000,0,3e-5
dcdirect,5000e3,0.47e3,680e3,20280e3,118.28e3,129930e3,757.84e3,2000,0.032,3e-5
conv,0,0,0,10140e3,59.14e3,64965e3,378.92e3,2000,0.016,0
ac_ohl,0,0.394e3,1187e3,1562e3,0,0,0,4000,0,3e-5


PARAMETERS - gentype:


...

,node,demand_avg,demand_ref,emission_cap
0,1,1,demand_GB,1.000000e+09
1,2,1,demand_DE,1.000000e+09
2,3,1,demand_NO,1.000000e+09


Generators:


,desc,type,node,pmax,pmax2,pmin,fuelcost,fuelcost_ref,pavg,inflow_fac,...,cost_scaling,p_maxNew,storage_cap,storage_price,storage_ini,storval_filling_ref,storval_time_ref,pump_cap,pump_efficiency,pump_deadband
0,UK,alt,1,100000,0,0,1,prices_GB,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DE,alt,2,100000,0,0,1,prices_DE,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NO,alt,3,100000,0,0,1,prices_NO_south,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Creyke Beck B 12,wind,67,600,0,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Creyke Beck B 34,wind,63,600,0,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Creyke Beck A,wind,66,1200,0,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Teeside A,wind,65,1200,0,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Teeside B,wind,40,1200,0,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Teeside C,wind,50,0,1200,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Teeside D,wind,60,0,1200,0,0,const,0,1,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Profiles:


,Unnamed: 0,const,wind0.3,wind1.2,prices_GB,prices_DE,prices_NO_south,demand_GB,demand_DE,demand_NO
0,0,1.0,0.133263,0.152860,40.325660,20.557245,17.464082,30894.265306,45562.857143,11369.275510
1,1,1.0,0.059113,0.072251,45.469202,40.045474,30.409368,30610.294737,45058.115789,13508.778947
2,2,1.0,0.481979,0.537927,68.675728,51.204426,34.059672,42443.327869,60922.245902,15877.196721
3,3,1.0,0.126789,0.163807,41.958703,48.453785,24.292617,38036.163551,61005.275701,12677.836449
4,4,1.0,0.070220,0.060419,79.024383,103.830000,98.178750,48562.708333,72108.750000,21567.000000
5,5,1.0,0.848609,0.855301,43.359131,18.881798,33.896404,33714.797753,46340.146067,17641.359551
6,6,1.0,0.122540,0.141419,59.488271,71.782716,35.467407,40888.876543,65985.950617,16435.802469
7,7,1.0,0.386009,0.487958,32.125343,15.354762,17.948254,22295.793651,36696.269841,10207.492063
8,8,1.0,0.123533,0.140922,54.896474,16.674906,23.239623,29817.339623,41973.943396,12004.094340
9,9,1.0,0.495796,0.551724,44.956040,48.793279,36.153852,44350.418033,63655.811475,18946.016393


In [470]:
grid_data.profiles.min()

Unnamed: 0             0.000000
const                  1.000000
wind0.3                0.039958
wind1.2                0.020603
prices_GB             32.125343
prices_DE           -254.000000
prices_NO_south        9.934286
demand_GB          22050.080460
demand_DE          30511.500000
demand_NO           9528.954023
dtype: float64

### Plotting input data

In [471]:
# A little trick to use powergama map plot 
# (need to split branches into ac and dc)
plot_data = copy.deepcopy(grid_data)
plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
plot_data.branch = plot_data.branch[plot_data.branch['capacity']>0]
mask_dc = plot_data.branch['type'].str.startswith('dc')
plot_data.dcbranch = plot_data.branch[mask_dc]
plot_data.branch = plot_data.branch[~mask_dc]

m=powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
    filename=None,branchtype="capacity",nodetype="area")
print("\nExisting capacity:")
m

Nodes...
AC branches...
DC branches...
Consumers...
Generators...

Existing capacity:


In [472]:
# A little trick to use powergama map plot 
# (need to split branches into ac and dc)
plot_data = copy.deepcopy(grid_data)
plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
mask_dc = plot_data.branch['type'].str.startswith('dc')
plot_data.dcbranch = plot_data.branch[mask_dc]
plot_data.branch = plot_data.branch[~mask_dc]

m=powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
    filename=None,branchtype="capacity",nodetype="area")
print("\nTransmission corridors considered in optimisation:")
m

Nodes...
AC branches...
DC branches...
Consumers...
Generators...

Transmission corridors considered in optimisation:


### Creating and solving MILP problem

In [474]:
sip = pgim.SipModel()
dict_data = sip.createModelData(grid_data,
    datafile=file_parameters,
    maxNewBranchNum=5,maxNewBranchCap=5000)
model = sip.createConcreteModel(dict_data) 

#Enable access to dual values
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Computing B and DA matrices...
Creating B and DA coefficients...


In [475]:
#opt = pyo.SolverFactory('gurobi',solver_io='python')
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, 
                    tee=True, #stream the solver output
                    keepfiles=False, #print the LP file for examination
                    symbolic_solver_labels=True) # use human readable names
print("=====================================",results['Solver'][0])

Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Nov 24 2015 

command line - c:\users\hsven\bin\cbc.exe -printingOptions all -import C:\Users\hsven\AppData\Local\Temp\tmpaz701yox.pyomo.lp -stat=1 -solve -solu C:\Users\hsven\AppData\Local\Temp\tmpaz701yox.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 15347 (-6582) rows, 12882 (-4727) columns and 47124 (-25248) elements
Statistics for presolved model
Original problem has 83 integers (50 of which binary)
Presolved problem has 51 integers (18 of which binary)
==== 12296 zero objective 555 different
==== absolute objective values 553 different
==== for integers 0 zero objective 27 different
==== for integers absolute objective values 27 different
===== end objective counts


Problem has 15347 rows, 12882 columns (586 with objective) and 47124 elements
There are 522 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 12864 of type 0.0->up, 0 of type lo->inf, 
0 of type

### Analyse results (plot and save to file)

In [520]:
#sip.saveDeterministicResults(model=model,
#    excel_file='det_result{}.xlsx'.format(scenario))
def myplot(grid):
    plot_data = copy.deepcopy(grid)
    plot_data.spreadNodeCoordinates(inplace=True,radius=0.04)
    mask_dc = ~plot_data.branch['type'].str.startswith('ac')
    plot_data.dcbranch = plot_data.branch[mask_dc]
    plot_data.branch = plot_data.branch[~mask_dc]
    m = powergama.plots.plotMap(pg_data=plot_data,pg_res=None,
        filename=None,branchtype="capacity",nodetype="area")
    return m
    
print("INPUT:")
m_input = myplot(grid_data)
display(m_input)

print("STAGE 1:")
grid_res = sip.extractResultingGridData(grid_data,model=model)
m_res1 = myplot(grid_res)
display(m_res1)

print("STAGE 2:")
grid_res2 = sip.extractResultingGridData(grid_data,model=model,stage=2)
m_res2 = myplot(grid_res2)
display(m_res2)

INPUT:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...


STAGE 1:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...


STAGE 2:
Nodes...
AC branches...
DC branches...
Consumers...
Generators...
